In [1]:
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
#pip install --user -U nltk


In [3]:
import pandas as pd
from datetime import datetime
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

[nltk_data] Error loading vader_lexicon: <urlopen error [WinError
[nltk_data]     10054] An existing connection was forcibly closed by
[nltk_data]     the remote host>


In [4]:
#

In [5]:
url = "https://economictimes.indiatimes.com/reliance-industries-ltd/stocksupdate/companyid-20316.cms"
request = requests.get(url) 
soup_page = BeautifulSoup(request.content, 'html.parser') 
  
quotes = []
  
search_div = soup_page.find('div', attrs = {'class':'all'}) 
search_class = search_div.findAll('div', attrs = {'class':'eachStory'})  
for chunk in search_class: 
    quote = {} 
    text_heading = chunk.find('div', attrs = {'class':'headingText'})
    text_date = chunk.find('div', attrs = {'class':'storyDate'})
    quote['date'] = text_date.time.text
    quote['text'] = text_heading.h3.text + '. ' + chunk.p.text
    quotes.append(quote)

In [6]:
scores = []
for quote in quotes:
    score = {}
    adj_date  = quote['date'].split(',')
    yr = adj_date[1].split(' ')
    if yr[0] != '':
        score['date'] = adj_date[0] + ' ' + yr[0]
        values = analyser.polarity_scores(quote['text'])
        score['value'] = values['compound']
        scores.append(score) 

In [7]:
lst = []
for scr in scores:
    tmp = {}
    strg = scr['date']
    dt = datetime.strptime(strg, '%d %b %Y')
    tmp['date'] = dt.strftime('%Y-%m-%d')   
    tmp['Sentiment'] = scr['value']
    lst.append(tmp)

dframe = pd.DataFrame(lst)
df = dframe.groupby("date").mean()

df.to_csv('1.csv', index = True) 

In [8]:
lst = []
for scr in scores:
    tmp = {}
    strg = scr['date']
    dt = datetime.strptime(strg, '%d %b %Y')
    tmp['Date'] = dt.strftime('%Y-%m-%d')   
    tmp['Sentiment'] = scr['value']
    lst.append(tmp)

dframe = pd.DataFrame(lst)
df = dframe.groupby('Date').mean()

df.to_csv('1.csv', index = True) 

In [9]:
len(quotes)

40